# Automatic Index Alignment

This notebook discusses **automatic index alignment**, a surprising, occasionally useful, and frustrating feature built into pandas. Automatic alignment of the index happens when operating on two pandas objects at the same time. Whether operating with two Series, two DataFrames, or one of each, automatic alignment of the index takes place first and then the operation completes.

## Adding two Series - Not as simple as it sounds
Adding two Series together should be a simple, and most of the time it is, but you can be in for quite a surprise if the indexes do not align. Let's create two identical Series. The `copy` method allows us to do this.

In [ ]:
import numpy as np
import pandas as pd
s1 = pd.Series(index=['a', 'b', 'c', 'd'], data=[0, 1, 2, 3])
s2 = s1.copy()

In [ ]:
s1

In [ ]:
s2

Note, that these are two distinct objects. If we wrote **`s2 = s1`**, we would not have created a new object, just two variable names that refer to the same object.

In [ ]:
s1 is s2

### Add the Series together
The Series have the same index and the same values. No surprises here.

In [ ]:
s1 + s2

### Create a new Series with index values in a different order
We create new Series **`s3`** below with the same index values but in a different position than **`s1`**

In [ ]:
s3 = pd.Series(index=['d', 'c', 'b', 'a'], data=[0, 1, 2, 3])
s3

### Add `s1` to `s3`

In [ ]:
s1 + s3

### What happened?
Pandas aligns the data first by the index and then completes the operation. Index 'a' aligns for both Series. In **`s1`** index 'a' labels value 3 and in **`s3`** it labels value 0. Added together they sum to 3. All the indexes align in this manner and all sum to 3.

## Adding a NumPy array to a Series
NumPy arrays have no index, just values and integer locations that refer to those values. NumPy arrays align by their integer location (which is what you would expect).

Let's create a simple array with integers 0 to 3 and add it to our Series from above. The index of the Series plays no role in the following operations.

In [ ]:
a = np.arange(4)
a

In [ ]:
s1 + a

In [ ]:
s3 + a

Adding the array to itself also aligns by integer location.

In [ ]:
a + a

### Adding arrays to Series - Must have same number of elements
For a successful array to Series addition to occur, they both need to have the same number of elements or else an error will occur.

In [ ]:
a = np.arange(5)
a

In [ ]:
try:
    s1 + a
except Exception as e:
    print(type(e), e)

## Adding Series that don't have the same index labels
Adding Series that do not have the same index labels is possible. In fact, adding two Series together will always complete (unless their values are incompatible - such as adding a number to a string).

In the following example, we have two Series of different lengths. **`s1`** has one more index label, **`d`**, that **`s2`** does not have. When we add them together, again the indexes align, except for the **`d`**. It has no matching index in **`s2`**. Pandas keeps this label in the returned Series but with a missing value.

Any label that does not match in the other Series is always kept and its associated value will always be missing.

In [ ]:
s1 = pd.Series(index=['a', 'b', 'c', 'd'], data=[0, 1, 2, 3])
s2 = pd.Series(index=['a', 'b', 'c'], data=[0, 1, 2])

In [ ]:
s1 + s2

### Missing index labels in each Series
If each of the Series have index labels that do not appear in the other, then they will both be kept in the result with missing values.

In [ ]:
s1 = pd.Series(index=['a', 'b', 'c', 'd'], data=[0, 1, 2, 3])
s2 = pd.Series(index=['a', 'b', 'c', 'e'], data=[0, 1, 2, 3])

In [ ]:
s1 + s2

## Adding Series with duplicate values in the index
A big surprise awaits when you add two Series that each share duplicated index labels. Take a look at both Series below. **`s1`** and **`s2`** each have 3 'a', index labels. **`s1`** has 3 'b', 4 'c' and 1 'd' index label while **`s2`** has 2 'b', 1 'c', 1 'e' labels.

Let's add them together to see what happens.

In [ ]:
s1 = pd.Series(index=['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'd'], data=np.arange(11))
s2 = pd.Series(index=['a', 'a', 'a', 'b', 'b', 'c', 'e'], data=np.arange(7))

In [ ]:
s1

In [ ]:
s2

In [ ]:
s1 + s2

In [ ]:
len(s1 + s2)

### 21 elements in resulting Series?

### A Cartesian product has taken place
Each index label 'a' from Series **`s1`** aligns with each index label 'a' from **`s2`**. There are 3 'a' labels in each which creates a total of 9 in the result. This is what is meant by a **Cartesian product**. All possible combinations of same index labels in each Series will have a result.

Similarly, Series **`s1`** has 3 'b' labels and **`s2`** has 2 'b' for a total of 6 in the result. Simply multiply the count of the labels in each Series together to get the total labels in the result. 

Label 'c' is found 4 times in **`s1`** and 1 time in **`s2`** for a total of 4 in the result. Labels 'd' and 'e' are unique to each Series so only occur once in the result with a missing value.

## An exception to Cartesian Product rule
If both Series share the exact same index labels then no Cartesian product will occur.

In [ ]:
s1 = pd.Series(index=['a', 'a', 'a', 'b', 'b'], data=np.arange(5))
s2 = pd.Series(index=['a', 'a', 'a', 'b', 'b'], data=np.arange(5))

In [ ]:
s1 + s2

But even if one index label is different than a Cartesian product will happen:

In [ ]:
s1 = pd.Series(index=['a', 'a', 'a', 'b', 'b'], data=np.arange(5))
s2 = pd.Series(index=['a', 'a', 'a', 'b', 'b', 'c'], data=np.arange(6))
s1 + s2

## Cartesian product still happens if order is not the same
Even if the index labels share the same number of occurrences in the Series, a Cartesian Product will still happen if the order is different. Below, **`s1`** and **`s2`** have the same number of 'a' and 'b' labels but have a different order for the 3rd and 4th labels.

In [ ]:
s1 = pd.Series(index=['a', 'a', 'b', 'a', 'b'], data=np.arange(5))
s2 = pd.Series(index=['a', 'a', 'a', 'b', 'b'], data=np.arange(5))
s1 + s2

## DataFrames align on both their index and columns

In [ ]:
df1 = pd.DataFrame(data={'first': np.arange(4), 'second': np.arange(4)}, index=['a', 'b', 'c', 'd'])
df2 = df1.copy()

Operations happen as expected whenever index and columns match exactly.

In [ ]:
df1 + df2

### DataFrame Index alignment
The label needs to be present in both DataFrames for a value to be computed or else it will be missing.

In [ ]:
df1 = pd.DataFrame(data={'first': np.arange(4), 'second': np.arange(4)}, index=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(data={'first': np.arange(4), 'second': np.arange(4)}, index=['a', 'b', 'c', 'e'])
df1

In [ ]:
df2

In [ ]:
df1 + df2

### When Columns do not align

In [ ]:
df1 = pd.DataFrame(data={'first': np.arange(4), 'second': np.arange(4)}, index=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(data={'first': np.arange(4), 'third': np.arange(4)}, index=['a', 'b', 'c', 'e'])
df1

In [ ]:
df2

In [ ]:
df1 + df2

## Cartesian Product over index and columns

In [ ]:
df1 = pd.DataFrame(data=np.random.rand(7, 5), 
                   index=['a', 'a', 'a', 'b', 'b', 'c', 'f'], 
                   columns=['first', 'first', 'second', 'second', 'third'])
df2 = pd.DataFrame(data=np.random.rand(8, 5), 
                   index=['a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                   columns=['first', 'first', 'first', 'second', 'second'])
(df1 + df2).shape

In [ ]:
df2

In [ ]:
df1 + df2